In [1]:
import pandas as pd
import re
import PyPDF2
import os
import json
import copy
import numpy as np
import requests

import tabula as tb

In [2]:
pdf_path = "2015_house_pdfs/Pelosi_Nancy_10010857.pdf"
pdf_path_joe = "2015_house_pdfs/Wilson_Joe_10012512.pdf"
pdf_to_empty = "2015_house_pdfs/Pelosi_Nancy_20002351.pdf"

In [3]:
pdf_to_empty = "2015_house_pdfs/Pelosi_Nancy_20002351.pdf"
if os.stat(pdf_to_empty).st_size < 2000:
    print("smaller")
else:
    print("larger")

smaller


In [4]:
def create_template(file_path):
    file = open(file_path, 'rb')
    readpdf = PyPDF2.PdfFileReader(file)
    total_pages = readpdf.numPages
    with open("pdf_reduced_template.tabula-template.json", "r") as jsonTemplate:
        data = json.load(jsonTemplate)
        page_template = data[3]
        # print(page_template)
        if total_pages == 1:
            data = data[:-1]
        else:
            for page_num in range(2, total_pages):
                tmp_page_template = copy.copy(page_template)
                tmp_page_template['page'] = page_num + 1
                # print(tmp_page_template)
                data.append(tmp_page_template)
            
        with open("pdf_tmp_template.tabula-template.json", 'w') as outfile:
            json.dump(data, outfile)
                
            
            

In [5]:
def scrape_pdf(file):
    create_template(file)
    template_path = 'pdf_tmp_template.tabula-template.json'
    return tb.read_pdf_with_template(file, template_path, stream=True)

In [6]:
db = scrape_pdf(pdf_path)

In [7]:
db_joe = scrape_pdf(pdf_path_joe)

In [70]:
pdf_db = pd.DataFrame(columns=['Member Name', 'Member District', 'Company', 'Ticker', 'Type', 'Date',
                      'Value Lower Bound','Value Upper Bound', 'Description', 'Link'])

In [9]:
db[7]

,asset owner,value of asset,income,Unnamed: 0,income tx. >
0,NaN,NaN,type(s),NaN,"$1,000?"
1,NaN,NaN,NaN,NaN,cdfeg
2,Account,NaN,NaN,NaN,NaN
3,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
4,Union Bank of California - Pelosi Trust Checki...,"$1 - $1,000",None,NaN,ecgdf
5,Account,NaN,NaN,NaN,NaN
6,"DESCRIPTION: San Francisco, CA",NaN,NaN,NaN,NaN
7,United Football League SP,"$5,000,001 -",None,NaN,fcedg
8,NaN,"$25,000,000",NaN,NaN,NaN
9,"LOCATION: Jacksonville/Duval, FL, US",NaN,NaN,NaN,NaN


In [10]:
"schedule b: transactions" in db[7]["asset owner"].str.lower().values

True

In [11]:
db[3].iloc[:, 0]

0                                                   NaN
1                                      Aristotle LLC SP
2         LOCATION: San Francisco/San Francisco, CA, US
3     DESCRIPTION: Ltd. Partnership Investment in Ar...
4               Attensity Group Inc. Preferred Stock SP
5     DESCRIPTION: Investment in Hosted Analytical S...
6                                  Auberge du Soleil SP
7                                                   NaN
8                     LOCATION: Rutherford/Napa, CA, US
9     DESCRIPTION: Ltd. Partnership Investment in Re...
10                Bank of America - Checking Account SP
11                       DESCRIPTION: San Francisco, CA
12                Bank of America - Checking Account JT
13                       DESCRIPTION: San Francisco, CA
14                Bank of America - Checking Account SP
15                          DESCRIPTION: Washington, DC
16            BF Enterprises Common Stock New (BFET) SP
17    DESCRIPTION: Asset name change during 2015

In [12]:
index = db[6].index[db[6]["asset owner"].str.lower() == "schedule b: transactions"].tolist()
index



[]

In [13]:
def find_start(df):
    for i in range(len(df)):
        index = df[i].index[df[i].iloc[:, 0].str.lower() == "schedule b: transactions"].tolist()
        if index != []:
            return i, index[0]

In [14]:
find_start(db_joe)

(3, 11)

In [67]:
db[0].iloc[2, 1]

'CA12'

In [16]:
pd.to_datetime(db[8].iloc[9, 1])

Timestamp('2015-12-30 00:00:00')

In [59]:
description_regex = r"^description: (.+)$"
stock_regex = r"(^.*)\s*\(([\w | .]{1,5})\)\s*$"
bounds_regex = r"^\$?([\d,]+)\s*-\s*\$?([\d,]*)$"
honorifics_regex = r"^(hon.?|mr.?|miss|mrs.?|ms.?|dr.?|professor.?|gen.?|)\s*([\w]+)\s([\w]*)\s*$"

In [63]:
result = re.match(honorifics_regex, 'hon. nancy pelosi')
result[2]+" " +result[3]

'nancy pelosi'

In [19]:
test = np.nan
test is np.nan

True

In [32]:
def generate_entry(df, df_index, row, regex_result):
    
    description_regex = r"^description: (.+)$"
    stock_regex = r"(^.*)\s*\(([\w | .]{1,5})\)\s*$"
    bounds_regex = r"^\$?([\d,]+)\s*-\s*\$?([\d,]*)$"
        
    company_name = regex_result[1].strip()
    ticker = regex_result[2].upper()
    
    
    owner_date = pd.to_datetime(df[df_index].iloc[row, 1])
    
    transaction_type = df[df_index].iloc[row, 2]
    
    bounds_regex_result = re.match(bounds_regex, df[df_index].iloc[row, 4])
    lower_bound = bounds_regex_result[1]
    upper_bound = ''
    
    if bounds_regex_result[2] == '':
        regex_result = re.match(bounds_regex, df[df_index].iloc[row+1, 4])
        upper_bound = regex_result[1]
    else:
        upper_bound = bounds_regex_result[2]
    
    
    description = None
        
    row += 1
    
    stock_info = str(df[df_index].iloc[row, 0]).lower()
    
    potential_new_stock = re.match(stock_regex, stock_info)
    while potential_new_stock is None:
        regex_result = re.match(description_regex, str(df[df_index].iloc[row, 0]).lower())
        if regex_result is not None:
            description = regex_result[1]
        row += 1
        
        stock_info = str(df[df_index].iloc[row, 0]).lower()
        potential_new_stock = re.match(stock_regex, stock_info)
        
    return {'Member Name': np.nan,
            'Member District': np.nan,
            'Company': company_name,
            'Ticker': ticker,
            'Type': transaction_type,
            'Date': owner_date,
            'Value Lower Bound' : lower_bound,
            'Value Upper Bound' : upper_bound,
            'Description' : description,
            'Link' : np.nan}
        

In [71]:
res = re.match(stock_regex, str(db[7].iloc[29, 0]).lower())
test_entry = generate_entry(db, 7, 29, res)


In [89]:
pd.concat([pdf_db, pd.Series(test_entry).to_frame().T], ignore_index=True)

,Member Name,Member District,Company,Ticker,Type,Date,Value Lower Bound,Value Upper Bound,Description,Link
0,NaN,NaN,"hertz global holdings, inc",HTZ,P,2015-07-15 00:00:00,"15,001","50,000",purchase of 42 call options with a strike pric...,NaN


In [90]:
def process_dataframe(df, link):
    
    pdf_db = pd.DataFrame(columns=['Member Name', 'Member District', 'Company', 'Ticker', 'Type', 'Date',
                      'Value Lower Bound','Value Upper Bound', 'Description', 'Link'])
    
    description_regex = r"^description: (.+)$"
    stock_regex = r"(^.*)\s*\(([\w | .]{1,5})\)\s*$"
    bounds_regex = r"^\$?([\d,]+)\s*-\s*\$?([\d,]*)$"
    honorifics_regex = r"^(hon.?|mr.?|miss|mrs.?|ms.?|dr.?|professor.?|gen.?|)\s*([\w]+)\s([\w]*)\s*$"

    name_regex_result = re.match(honorifics_regex, df[0].iloc[0, 1].lower())
    name = name_regex_result[2] + " " + name_regex_result[3]
                             
    district = df[0].iloc[2, 1]            
                             
    df_index, row = find_start(df)
    
    
    row += 1
    stock_info = str(df[df_index].iloc[row, 0]).lower()
    while stock_info != "schedule c: earned income":
        regex_result = re.match(stock_regex, stock_info)
        if regex_result is not None:
            result_dict = generate_entry(df, df_index, row, regex_result)
            result_dict['Member Name'] = name
            result_dict['Member District'] = district
            result_dict['Link'] = link
            pdf_db = pd.concat([pdf_db, pd.Series(result_dict).to_frame().T], ignore_index=True)
        
        if len(df[df_index].index)-1 == row:
            row = 0
            df_index += 1
        else:
            row += 1
        
        stock_info = str(df[df_index].iloc[row, 0]).lower()
    
    return pdf_db

In [108]:
db_joe[3]

,asset,owner value of asset,Unnamed: 0,income,income tx. >,Unnamed: 1
0,NaN,NaN,NaN,type(s),"$1,000?",NaN
1,"LOCATION: West Columbia, SC, US",NaN,NaN,NaN,NaN,NaN
2,"Moseley and Wilson Partnership, 515 E. Main St.,","$50,001 -",NaN,Rent,"$5,001 -",efcgd
3,"Lexington, SC","$100,000",NaN,NaN,"$15,000",NaN
4,"LOCATION: lexington, SC, US",NaN,NaN,NaN,NaN,NaN
5,"Moseley and Wilson Partnership, 634-640 Sunset...","$250,001 -",NaN,Rent,"$15,001 -",cdegf
6,"West Columbia, SC","$500,000",NaN,NaN,"$50,000",NaN
7,"LOCATION: West Columbia, SC, US",NaN,NaN,NaN,NaN,NaN
8,"Moseley and Wilson Partnership, 922 Sunset Blvd.,","$100,001 -",NaN,Rent,"$5,001 -",dcegf
9,"West Columbia, SC","$250,000",NaN,NaN,"$15,000",NaN


In [91]:
process_dataframe(db, None)

TypeError: 'NoneType' object is not subscriptable